# Task 5
Record an original video of a subject in a stationary background (No green screens).

    a. Change the original background with another background.
    b. Show the optical flow of the video

## Prerequisite
Before running below code make sure to install packages in 'requirements.txt' file.

Ref: https://youtu.be/p9eNXa_8j-k

In [ ]:
# !pip install cvzone

In [ ]:
# !pip install mediapipe

In [18]:
# !pip install "opencv-python-headless<4.3"

In [1]:
# Importing packages

import time
import numpy as np
## cv2 for processing image files
import cv2 

## importing cvzone; this package used to replace static background in the video
import cvzone

## importing SelfiSegmentation class from cvzone package; this class will helps to segment object and background
## in the frame image
from cvzone.SelfiSegmentationModule import SelfiSegmentation

In [2]:
# Code for recording and saving video capture

## value '0' will return ON the default webcam on computer for recording.
cap = cv2.VideoCapture(0)

## defining the codec 
fourcc = cv2.VideoWriter_fourcc(*'XVID')
## create VideoWriter object to save every frame into 'output.mp4' with shape of 640, 480. 
out = cv2.VideoWriter('output.mp4', fourcc, 20.0, (640, 480))

# Below loop will show recording scene and save the frame
## loop runs if capturing has been initialized.
while cap.isOpened():
    ## reads frames from a camera
    status, frame = cap.read()
    # status checks presence of each frame
    
    ## check if frame is present
    if status == True:
        ## write the frame to 'out' object
        out.write(frame)
        ## output a window showing the video stream as individual frame
        cv2.imshow('frame', frame)

        ## Wait for 'q' key to stop the program
        if cv2.waitKey(1) == ord('q'):
            break
    
    ## if there is no any frame, will come out of loop
    else:
        break

## Close the window / Release webcam
cap.release()
## After we release our webcam, we also release/end the 'out' object
out.release()
## clearing any associated memory space
cv2.destroyAllWindows()

In [5]:
# Here, we will change the background of saved/recorded video

## importing video file
vid = cv2.VideoCapture('output.mp4')

## choosing the backgroung image
bg_img = cv2.imread('bg_image.jpg')
## Reshaping the image to that of video frame
bg_img_rez = cv2.resize(bg_img, (640, 480))
## creating a SelfiSegmentation object  
seg = SelfiSegmentation()

# Below loop will show recorded scene with replaced background
## loop runs if video has been initialized.
while vid.isOpened():
    ## reads frames from video
    status, frame = vid.read()
    ## status checks presence of each frame
    
    ## check if frame is present
    if status ==True:
        ## Here, we pass our background image, frame as an argument value into 'removeBG' method of 'seg' object. 
        vid_rmbg = seg.removeBG(frame, bg_img_rez, threshold=0.8)
        ## it will segment frame and replace background with image 'bg_img_rez', threshold can be change to modify degree of
        ## background removal
        
        ## output a window showing the video stream with new background as individual frame
        cv2.imshow('Backgroung Removed', vid_rmbg)

        ## Wait for 'q' key to stop the program
        if cv2.waitKey(1) == ord('q'):
            break
    
    ## if there is no any frame, will come out of loop
    else:
        break

## Close the window / Release video
vid.release()
## clearing any associated memory space
cv2.destroyAllWindows()

### Optical flow of the video

In [6]:
def draw_flow(img, op_flow, step=16):
    hight, width = img.shape[:2]
    y, x = np.mgrid[step/2:hight:step, step/2:width:step].reshape(2,-1).astype(int)
    fx, fy = op_flow[y,x].T

    lines = np.vstack([x, y, x-fx, y-fy]).T.reshape(-1, 2, 2)
    lines = np.int32(lines + 0.5)

    img_bgr = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    cv2.polylines(img_bgr, lines, 0, (0, 255, 0))

    for (x1, y1), (_x2, _y2) in lines:
        cv2.circle(img_bgr, (x1, y1), 1, (0, 255, 0), -1)

    return img_bgr

In [7]:
def draw_hsv_flow(op_flow):
    hight, width = img.shape[:2]
    fx, fy = op_flow[:,:,0], op_flow[:,:,1]

    ang = np.arctan2(fy, fx) + np.pi
    v = np.sqrt(fx*fx+fy*fy)

    hsv = np.zeros((hight, width, 3), np.uint8)
    hsv[...,0] = ang*(180/np.pi/2)
    hsv[...,1] = 255
    hsv[...,2] = np.minimum(v*4, 255)
    bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

    return bgr

In [8]:
vid = cv2.VideoCapture('output.mp4')
_, prev = vid.read()
prev_gray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)


while True:
    _, img = vid.read()
    curr_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    optical_flow = cv2.calcOpticalFlowFarneback(prev_gray, curr_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    prev_gray = curr_gray

    cv2.imshow('flow', draw_flow(curr_gray, optical_flow))
    cv2.imshow('flow HSV', draw_hsv_flow(optical_flow))


    key = cv2.waitKey(5)
    if key == ord('q'):
        break

vid.release()
cv2.destroyAllWindows()

Ref: https://www.youtube.com/watch?v=WrlH5hHv0gE